In [2]:
import numpy as np
import h5py
from scipy.sparse import coo_matrix, csr_matrix
import pandas as pd
import scipy.io
import scipy.sparse as sparse
import anndata

/tmp/ipykernel_96565/962953863.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [6]:
input_ad = '../../../results_scbasset/promotor_final/data/filtered/cage_promotor_ad.h5ad'
ad = anndata.read_h5ad(input_ad)

In [34]:
ad.var

,gene_ids,feature_types,genome,chr,start,end,strand,n_cells
chr1:629175-629265:+,chr1:629175-629265:+,Peaks,NA,chr1,629175,629265,+,6
chr1:629806-629930:+,chr1:629806-629930:+,Peaks,NA,chr1,629806,629930,+,65
chr1:630223-630315:+,chr1:630223-630315:+,Peaks,NA,chr1,630223,630315,+,26
chr1:631294-631361:+,chr1:631294-631361:+,Peaks,NA,chr1,631294,631361,+,8
chr1:632231-632378:+,chr1:632231-632378:+,Peaks,NA,chr1,632231,632378,+,24
...,...,...,...,...,...,...,...,...
chrX:155612891-155612963:-,chrX:155612891-155612963:-,Peaks,NA,chrX,155612891,155612963,-,96
chrY:5573461-5573465:+,chrY:5573461-5573465:+,Peaks,NA,chrY,5573461,5573465,+,80
chrY:13286618-13286778:+,chrY:13286618-13286778:+,Peaks,NA,chrY,13286618,13286778,+,95
chrY:5337904-5337905:-,chrY:5337904-5337905:-,Peaks,NA,chrY,5337904,5337905,-,75


In [52]:
chro_to_exclude = 'chr1'
ad.shape[1]

24275

In [45]:
test

array([    0,     1,     2, ..., 24272, 24273, 24274])

In [39]:
indices = ad.var['chr'] == chro_to_exclude

ligne_indices = np.where(indices)[0] + 1

In [40]:
ligne_indices 

array([   1,    2,    3, ..., 2643, 2644, 2645])

In [71]:
def split_train_test_val(ad, exclude_chr,  seed=10, train_ratio=0.9 ):
    np.random.seed(seed)
    ids = np.arange(ad.shape[1])

    test_val_ids = np.where(ad.var['chr'] == exclude_chr)[0]
    percentage_test_val = (len(test_val_ids)*100/len(ids))
    print (percentage_test_val)

    if percentage_test_val >(1-train_ratio):
        print('Chromosome ids are not large enough')

    train_ids = np.setdiff1d(ids, test_val_ids)

    val_ids = np.random.choice(test_val_ids,
        int(len(test_val_ids) / 2),
        replace=False,
    )
    test_ids = np.setdiff1d(test_val_ids, val_ids)
    return train_ids, test_ids, val_ids

In [72]:
exclude_chr = 'chr19'
train_ids, test_ids, val_ids = split_train_test_val(ad, exclude_chr,  seed=10)

6.047373841400618
Chromosome ids are not large enough


In [68]:
print(len(train_ids)* 100/ad.shape[1])
print(len(test_ids)* 100 /ad.shape[1])
print(len(val_ids)* 100 /ad.shape[1])

93.95262615859939
3.023686920700309
3.023686920700309


In [51]:
test_ids

array([6459, 6461, 6462, 6463, 6464, 6467, 6468, 6472, 6473, 6474, 6476,
       6477, 6478, 6480, 6482, 6484, 6485, 6487, 6491, 6495, 6497, 6499,
       6501, 6503, 6505, 6508, 6509, 6513, 6514, 6517, 6518, 6520, 6521,
       6522, 6524, 6525, 6529, 6530, 6531, 6536, 6538, 6540, 6542, 6545,
       6548, 6549, 6550, 6552, 6555, 6556, 6557, 6560, 6562, 6564, 6565,
       6566, 6569, 6570, 6571, 6572, 6576, 6577, 6578, 6586, 6587, 6588,
       6589, 6593, 6594, 6595, 6596, 6603, 6604, 6605, 6607, 6608, 6609,
       6613, 6615, 6617, 6619, 6621, 6623, 6624, 6627, 6629, 6637, 6639,
       6641, 6644, 6645, 6646, 6648, 6657, 6658, 6659, 6660, 6663, 6664,
       6665, 6666, 6667, 6668, 6669, 6671, 6673, 6674, 6678, 6680, 6683,
       6685, 6686, 6690, 6691, 6695, 6697, 6698, 6700, 6702, 6705, 6706,
       6710, 6715, 6716, 6717, 6722, 6724, 6725, 6726, 6727, 6728, 6729,
       6731, 6734, 6738, 6740, 6741, 6742, 6745, 6747, 6748, 6749, 6750,
       6755, 6756, 6760, 6762, 6765, 6767, 6769, 67

In [23]:
matrix='../../../results_scbasset/enhancer_final/data/count_matrix_enhancer.h5'
#matrix = '../../../scBasset/data/multiome_pbmc/pbmc_granulocyte_sorted_3k_filtered_feature_bc_matrix.h5'
output = '../../../results_scbasset/enhancer_final/data/test.csv'

### Recreer un csv à partir à partir de la count matrix

In [24]:
import h5py
import pandas as pd

def h5_to_csv(input_file, output_file):
    with h5py.File(input_file, 'r') as f:
        # Récupération des données du groupe 'matrix'
        matrix_group = f['matrix']
        barcodes = matrix_group['barcodes'][:]
        data = matrix_group['data'][:]
        shape = matrix_group['shape'][:]
        shape = (shape[1], shape[0])
        
        indices = matrix_group['indices'][:]
        indptr = matrix_group['indptr'][:]
        
        # Construction de la matrice CSR
        from scipy.sparse import csr_matrix
        data_matrix = csr_matrix((data, indices, indptr), shape=shape)
        
        # Récupération des données du groupe 'features'
        features_group = matrix_group['features']
        all_tag_keys = features_group['_all_tag_keys'][:]
        feature_type = features_group['feature_type'][:]
        genome = features_group['genome'][:]
        feature_id = features_group['id'][:]
        interval = features_group['interval'][:]
        name = features_group['name'][:]
        
        # Création d'un DataFrame avec les données
        df = pd.DataFrame(data_matrix.todense(), index=barcodes, columns=name)
        
        # Écriture du DataFrame dans un fichier CSV
        df.to_csv(output_file, index_label='barcodes')

# Utilisation de la fonction pour convertir le fichier h5 en CSV
input_file = matrix
output_file = output
h5_to_csv(input_file, output_file)

In [29]:
#matrix = '../../../scBasset/examples/PBMC_multiome/atac_ad.h5ad'
matrix = '../../../results_scbasset/enhancer_final/data/not_filtered/cage_enhancer_ad.h5ad'


### recreer le csv à partir du anndata

In [31]:
import h5py
import pandas as pd

def h5ad_to_csv(input_file, output_file):
    with h5py.File(input_file, 'r') as f:
        # Récupération des données des groupes nécessaires
        X_data = f['X']['data'][:]
        X_indices = f['X']['indices'][:]
        X_indptr = f['X']['indptr'][:]

        
        obs_index = f['obs']['_index'][:]
        obs_n_genes = f['obs']['n_genes'][:]
        
        var_index = f['var']['_index'][:]
        var_chr_categories = f['var']['chr']['categories'][:]
        var_chr_codes = f['var']['chr']['codes'][:]
        var_end = f['var']['end'][:]
        var_feature_types_categories = f['var']['feature_types']['categories'][:]
        var_feature_types_codes = f['var']['feature_types']['codes'][:]
        var_gene_ids = f['var']['gene_ids'][:]
        var_genome_categories = f['var']['genome']['categories'][:]
        var_genome_codes = f['var']['genome']['codes'][:]
        var_n_cells = f['var']['n_cells'][:]
        var_start = f['var']['start'][:]
        
        # Vérification que toutes les variables ont la même longueur
        lengths = [len(var_index), len(var_chr_categories), len(var_chr_codes),
                   len(var_end), len(var_feature_types_categories),
                   len(var_feature_types_codes), len(var_gene_ids),
                   len(var_genome_categories), len(var_genome_codes),
                   len(var_n_cells), len(var_start)]
        if len(set(lengths)) != 1:
            # Affichage des longueurs des arrays
            print("Longueurs des arrays :")
            print("var_index:", len(var_index))
            print("var_chr_categories:", len(var_chr_categories))
            print("var_chr_codes:", len(var_chr_codes))
            print("var_end:", len(var_end))
            print("var_feature_types_categories:", len(var_feature_types_categories))
            print("var_feature_types_codes:", len(var_feature_types_codes))
            print("var_gene_ids:", len(var_gene_ids))
            print("var_genome_categories:", len(var_genome_categories))
            print("var_genome_codes:", len(var_genome_codes))
            print("var_n_cells:", len(var_n_cells))
            print("var_start:", len(var_start))
            raise ValueError("Les arrays n'ont pas la même longueur.")
        
        # Création d'un DataFrame pour les données des observations (obs)
        obs_df = pd.DataFrame({'index': obs_index, 'n_genes': obs_n_genes})
        
        # Création d'un DataFrame pour les données des variables (var)
        var_df = pd.DataFrame({'index': var_index,
                               'chr_categories': var_chr_categories,
                               'chr_codes': var_chr_codes,
                               'end': var_end,
                               'feature_types_categories': var_feature_types_categories,
                               'feature_types_codes': var_feature_types_codes,
                               'gene_ids': var_gene_ids,
                               'genome_categories': var_genome_categories,
                               'genome_codes': var_genome_codes,
                               'n_cells': var_n_cells,
                               'start': var_start})
        
        # Écriture des DataFrames dans un fichier CSV
        obs_df.to_csv(output_file + '_obs.csv', index=False)
        var_df.to_csv(output_file + '_var.csv', index=False)
        
        # Reconstruction de la matrice X à partir des indices et des pointeurs
        data_matrix = []
        for i in range(len(X_indptr) - 1):
            row_indices = X_indices[X_indptr[i]:X_indptr[i + 1]]
            row_data = X_data[X_indptr[i]:X_indptr[i + 1]]
            row = [0] * len(var_index)
            for j, idx in enumerate(row_indices):
                row[idx] = row_data[j]
            data_matrix.append(row)
        
        # Création d'un DataFrame pour la matrice X
        X_df = pd.DataFrame(data_matrix, index=obs_index, columns=var_index)
        
        # Écriture de la matrice X dans un fichier CSV
        X_df.to_csv(output_file + '_X.csv')

# Utilisation de la fonction pour convertir le fichier h5ad en CSV
input_file = matrix
output_file = '../../../results_scbasset/enhancer_final/data/not_filtered/test_tuto.csv'
h5ad_to_csv(input_file, output_file)


KeyError: "Unable to synchronously open object (object 'shape' doesn't exist)"

# SCAFE

In [9]:
matrix='../../../results_scbasset/data/SCAFE/Malignant_cells_distal_cre.mtx
column='../../../results_scbasset/data/SCAFE/colnames_Malignant_cells_distal_cre.txt'
row='../../../results_scbasset/data/SCAFE/rownames_Malignant_cells_distal_cre.txt'

In [5]:
# Chemin vers votre fichier .mtx
fichier_mtx = matrix

# Lire le fichier .mtx et le stocker comme une matrice creuse
matrice_creuse = scipy.io.mmread(fichier_mtx)

# Convertir la matrice creuse en format CSR
matrice_csr = matrice_creuse.tocsr()

In [6]:
matrice_csr.shape

(153373, 104042)

In [3]:
# Define functions to read each type of file
def read_mtx(matrix_file):
    with open(matrix_file, 'r') as f:
        # Skip headers
        for _ in range(2):
            next(f)
        data, row, col = [], [], []
        for line in f:
            r, c, d = map(int, line.split())
            row.append(r - 1)  # adjust for zero-based indexing
            col.append(c - 1)  # adjust for zero-based indexing
            data.append(d)
    return row, col, data

def read_names(file):
    with open(file, 'r') as f:
        return [line.strip() for line in f]

In [4]:
row_indices, col_indices, data = read_mtx(matrix)
sparse_matrix = coo_matrix((data, (row_indices, col_indices)))
csr_matrix_data = sparse_matrix.tocsr()
    
# Read the column and row names
barcodes_list = np.array(read_names(column), dtype='S')  # Convert to bytes for h5py compatibility
features_list = np.array(read_names(row), dtype='S')  # Same as above
    
# Assume some feature type and genome data for the example, adjust as necessary
feature_types_data = np.array(['gene_expression'] * len(features_list), dtype='S')
genome_data = np.array(['GRCh38'] * len(features_list), dtype='S')

In [14]:
len(features_list)

153373

In [12]:
grp_matrix = f.create_group('matrix')
        
        # Write the barcodes (column names in this context)
grp_matrix.create_dataset('barcodes', data=barcodes_list)
        
        # Write the count matrix CSR components
grp_matrix.create_dataset('data', data=csr_matrix_data.data)
grp_matrix.create_dataset('indices', data=csr_matrix_data.indices)
grp_matrix.create_dataset('indptr', data=csr_matrix_data.indptr)
        

NameError: name 'f' is not defined